In [ ]:
import spacy
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
import json
import tqdm
import os

nlp = spacy.load('en_core_web_sm')

text = "When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of the company took him seriously."
doc = nlp(text)

for entity in doc.ents:
    print(entity.text, entity.label_)

Sebastian Thrun PERSON
2007 DATE


In [ ]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
TRAIN_DATA = [('The F15 aircraft uses a lot of fuel', {'entities': [(4, 7, 'aircraft')]}),
 ('did you see the F16 landing?', {'entities': [(16, 19, 'aircraft')]}),
 ('how many missiles can a F35 carry', {'entities': [(24, 27, 'aircraft')]}),
 ('is the F15 outdated', {'entities': [(7, 10, 'aircraft')]}),
 ('does the US still train pilots to dog fight?',
  {'entities': [(0, 0, 'aircraft')]}),
 ('how long does it take to train a F16 pilot',
  {'entities': [(33, 36, 'aircraft')]}),
 ('how much does a F35 cost', {'entities': [(16, 19, 'aircraft')]}),
 ('would it be possible to steal a F15', {'entities': [(32, 35, 'aircraft')]}),
 ('who manufactures the F16', {'entities': [(21, 24, 'aircraft')]}),
 ('how many countries have bought the F35',
  {'entities': [(35, 38, 'aircraft')]}),
 ('is the F35 a waste of money', {'entities': [(7, 10, 'aircraft')]})]
 #train_data=[(text,{"entities":[(start_char,end_char,label)]}
 #table entity : entity ->id_intent ManytoOne


In [ ]:
#Conversion to .spacy format
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

#nlp = spacy.blank("en") # load a new spacy model
nlp = spacy.load("en_core_web_sm") # load other spacy model

db = DocBin() # create a DocBin object

for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)
db.to_disk("./train.spacy") # save the docbin object

100%|██████████| 11/11 [00:00<00:00, 1310.27it/s]

Skipping entity


In [ ]:
#génération du train_data sous le format suivant: [(text,{"entities":[(start_char,end_char,label)]}
train_data = []
nlp = English()
patterns = [{"label":"fruit","pattern":"orange"},{"label":"vegetable","pattern":"pepper"}]
ruler = nlp.add_pipe("entity_ruler")
ruler.add_patterns(patterns)
doc=nlp("i have eaten an orange and a pepper")
entities=[]
for entity in doc.ents:
    print(entity.text, entity.label_)
    entities.append((entity.start_char,entity.end_char,entity.label_))
train_data.append(("i have eaten an orange and a pepper",{'entities':entities}))
train_data

orange fruit
pepper vegetable


[('i have eaten an orange and a pepper',
  {'entities': [(16, 22, 'fruit'), (29, 35, 'vegetable')]})]

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy
#trainning  model

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-07-22 13:06:09,695] [INFO] Set up nlp object from config
[2022-07-22 13:06:09,713] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-07-22 13:06:09,718] [INFO] Created vocabulary
[2022-07-22 13:06:10,350] [WARNING] [W112] The model specified to use for initial vectors (en_core_web_sm) has no vectors. This is almost certainly a mistake.
[2022-07-22 13:06:10,353] [INFO] Added vectors: en_core_web_sm
[2022-07-22 13:06:10,354] [INFO] Finished initializing nlp object
[2022-07-22 13:06:10,636] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00

In [ ]:
nlp1 = spacy.load(r"./output/model-best") #load the best model

In [ ]:
doc = nlp1("is A36 is bought by tunisia ")

In [ ]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)